## Install the YOLOV5 folder

In [1]:
import os
import shutil
from codecarbon import OfflineEmissionsTracker

if not os.path.exists("yolov5"):
    !git clone https://github.com/ultralytics/yolov5
else:
    print("The yolov5 repository has been cloned.")
pathInicial=os.getcwd() 
os.chdir("yolov5")
import torch
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

The yolov5 repository has been cloned.
Setup complete. Using torch 1.12.1+cu113 _CudaDeviceProperties(name='Quadro RTX 4000', major=7, minor=5, total_memory=7982MB, multi_processor_count=36)


# We load the roboflow data of the wheelchair.

In [2]:
if not os.path.exists("wheelchair-detection-1"):
    from roboflow import Roboflow
    rf = Roboflow(api_key="GTFcmyUsIPX2cUOUUP8y")
    project = rf.workspace("2458761304-qq-com").project("wheelchair-detection")
    dataset = project.version(1).download("yolov5")
else:
    print("Good,¡The data is already load!")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to wheelchair-detection-1 in yolov5pytorch:: 100%|█| 1037/1037 [00:00<00:00, 13586.48it/s


# Decide if training with the full train dataset or if reducing dataset

You must select one of the possible methods:

- NONE
- SRS
- DES
- MMS
- NRMD
- RKM
- PHL
- PRD
- FES

You must select a reduction rate, for example if you want a reduction rate of 75%, you have to insert 0.25 in perc, that is to remain only with the 25% of full data.

First, we are going to create a neural network with all the train data and then one with a subset of the 50% of the data.

## Train with full train data

In [5]:
os.chdir(pathInicial)
method = "NONE"
perc=0.25
!python ReductionDatasetRoboflow_yolov5.py --name {method} --perc {perc}

2024-09-11 10:22:47.581193: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-11 10:22:47.598616: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-11 10:22:47.603634: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
You have not selected any method, so you are going to train with the complete training set.
Number Original Files: 463
The training set has a size of:  463


We updated the data yaml, in order to correctly run the training and evaluation of the model with the reduced data.

In [6]:
os.chdir("yolov5")

import yaml
with open("wheelchair-detection-1" + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [7]:
%%writetemplate wheelchair-detection-1/data.yaml
names:
- people
- peopleWithWheelchair
nc: 2
roboflow:
  license: CC BY 4.0
  project: wheelchair-detection
  url: https://universe.roboflow.com/2458761304-qq-com/wheelchair-detection/dataset/1
  version: 1
  workspace: 2458761304-qq-com
test: wheelchair-detection-1/test/images
train: wheelchair-detection-1/train/images
val: wheelchair-detection-1/test/images

In [8]:
os.chdir(pathInicial + '/yolov5')
seed = 0
comando = f"python train.py --img 640 --batch 16 --epochs 100 --data wheelchair-detection-1/data.yaml --weights yolov5m.pt --freeze 10 --name yolov5_results_roboflow_GAM24_fullTrainSet --cache --seed {seed}"
!{comando}

WARNING ⚠️ Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View settings with 'yolo settings' or at '/home/vtoscano/.config/Ultralytics/settings.yaml'
Update settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
train: weights=yolov5m.pt, cfg=, data=wheelchair-detection-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5_results_roboflow_GAM24_fullTrainSet, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[10], sav

Make the inference in the video we have choose for the M24 GA of REXASI-PRO

In [9]:
comando = f'python detect.py --weights runs/train/yolov5_results_roboflow_GAM24_fullTrainSet/weights/best.pt --img 640 --conf 0.4 --source ../GAM24_video.mp4 --name GAM24_fullTrainSet'
!{comando}

detect: weights=['runs/train/yolov5_results_roboflow_GAM24_fullTrainSet/weights/best.pt'], source=../GAM24_video.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=GAM24_fullTrainSet, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-365-g12b577c8 Python-3.9.19 torch-1.12.1+cu113 CUDA:0 (Quadro RTX 4000, 7982MiB)

Fusing layers... 
Model summary: 212 layers, 20856975 parameters, 0 gradients, 47.9 GFLOPs
video 1/1 (1/664) /home/vtoscano/VToscano/GA_M24/GAM24_video.mp4: 384x640 5 peoples, 1 peopleWithWheelchair, 14.0ms
video 1/1 (2/664) /home/vtoscano/VToscano/GA_M24/GAM24_video.mp4: 384x640 5 peoples, 1 peopleWithWheelchair, 13.7ms
video 1/1 (3/664) /home

## Train with a subset of the 50% of the data

In [11]:
os.chdir(pathInicial)
method = "SRS"
perc=0.5
!python ReductionDatasetRoboflow_yolov5.py --name {method} --perc {perc}

2024-09-11 10:41:04.259096: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-11 10:41:04.276002: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-11 10:41:04.281080: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Selected Method:  SRS
Selected reduction rate:  0.5
There are 463 images in the path yolov5/wheelchair-detection-1/train
YOLOv5 🚀 2024-2-29 Python-3.9.19 torch-1.12.1+cu113 CUDA:0 (Quadro RTX 4000, 7982MiB)

100%|██████████████████████████████████████| 40.8M/40.8M [00:01<00:00, 38.7MB/s]

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradi

In [14]:
os.chdir(pathInicial + '/yolov5')
seed = 0
comando = f"python train.py --img 640 --batch 16 --epochs 100 --data wheelchair-detection-1/data.yaml --weights yolov5m.pt --freeze 10 --name yolov5_results_roboflow_GAM24_subset --cache --seed {seed}"
!{comando}

train: weights=yolov5m.pt, cfg=, data=wheelchair-detection-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5_results_roboflow_GAM24_subset, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[10], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-365-g12b577c8 Python-3.9.19 torch-1.12.1+cu113 CUDA:0 (Quadro RTX 4000, 7982MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum

In [15]:
comando = f'python detect.py --weights runs/train/yolov5_results_roboflow_GAM24_subset/weights/best.pt --img 640 --conf 0.4 --source ../GAM24_video.mp4 --name GAM24_subset'
!{comando}

detect: weights=['runs/train/yolov5_results_roboflow_GAM24_subset/weights/best.pt'], source=../GAM24_video.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=GAM24_subset, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-365-g12b577c8 Python-3.9.19 torch-1.12.1+cu113 CUDA:0 (Quadro RTX 4000, 7982MiB)

Fusing layers... 
Model summary: 212 layers, 20856975 parameters, 0 gradients, 47.9 GFLOPs
video 1/1 (1/664) /home/vtoscano/VToscano/GA_M24/GAM24_video.mp4: 384x640 5 peoples, 1 peopleWithWheelchair, 14.0ms
video 1/1 (2/664) /home/vtoscano/VToscano/GA_M24/GAM24_video.mp4: 384x640 5 peoples, 1 peopleWithWheelchair, 13.6ms
video 1/1 (3/664) /home/vtoscano/VT